# Tree Methods Consulting Project 

You've been hired by a dog food company to try to predict why some batches of their dog food are spoiling much quicker than intended! Unfortunately this Dog Food company hasn't upgraded to the latest machinery, meaning that the amounts of the five preservative chemicals they are using can vary a lot, but which is the chemical that has the strongest effect? The dog food company first mixes up a batch of preservative that contains 4 different preservative chemicals (A,B,C,D) and then is completed with a "filler" chemical. The food scientists beelive one of the A,B,C, or D preservatives is causing the problem, but need your help to figure out which one!
Use Machine Learning with RF to find out which parameter had the most predicitive power, thus finding out which chemical causes the early spoiling! So create a model and then find out how you can decide which chemical is the problem!

* Pres_A : Percentage of preservative A in the mix
* Pres_B : Percentage of preservative B in the mix
* Pres_C : Percentage of preservative C in the mix
* Pres_D : Percentage of preservative D in the mix
* Spoiled: Label indicating whether or not the dog food batch was spoiled.
___

**Think carefully about what this problem is really asking you to solve. While we will use Machine Learning to solve this, it won't be with your typical train/test split workflow. If this confuses you, skip ahead to the solution code along walk-through!**
____

# Good Luck!

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Random_forest_Project').getOrCreate()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1524078363246_0005,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
from pyspark.ml.classification import RandomForestClassifier
#from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [3]:
# Load data
data_df = spark.read.csv("hdfs:///...dog_food.csv",
                     inferSchema = True, header = True)

In [10]:
data_df.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)

In [11]:
data_df.describe().show()

+-------+------------------+------------------+------------------+------------------+-------------------+
|summary|                 A|                 B|                 C|                 D|            Spoiled|
+-------+------------------+------------------+------------------+------------------+-------------------+
|  count|               490|               490|               490|               490|                490|
|   mean|  5.53469387755102| 5.504081632653061| 9.126530612244897| 5.579591836734694| 0.2857142857142857|
| stddev|2.9515204234399057|2.8537966089662063|2.0555451971054275|2.8548369309982857|0.45221563164613465|
|    min|                 1|                 1|               5.0|                 1|                0.0|
|    max|                10|                10|              14.0|                10|                1.0|
+-------+------------------+------------------+------------------+------------------+-------------------+

In [12]:
data_df.head()

Row(A=4, B=2, C=12.0, D=3, Spoiled=1.0)

In [13]:
data_df.show()

+---+---+----+---+-------+
|  A|  B|   C|  D|Spoiled|
+---+---+----+---+-------+
|  4|  2|12.0|  3|    1.0|
|  5|  6|12.0|  7|    1.0|
|  6|  2|13.0|  6|    1.0|
|  4|  2|12.0|  1|    1.0|
|  4|  2|12.0|  3|    1.0|
| 10|  3|13.0|  9|    1.0|
|  8|  5|14.0|  5|    1.0|
|  5|  8|12.0|  8|    1.0|
|  6|  5|12.0|  9|    1.0|
|  3|  3|12.0|  1|    1.0|
|  9|  8|11.0|  3|    1.0|
|  1| 10|12.0|  3|    1.0|
|  1|  5|13.0| 10|    1.0|
|  2| 10|12.0|  6|    1.0|
|  1| 10|11.0|  4|    1.0|
|  5|  3|12.0|  2|    1.0|
|  4|  9|11.0|  8|    1.0|
|  5|  1|11.0|  1|    1.0|
|  4|  9|12.0| 10|    1.0|
|  5|  8|10.0|  9|    1.0|
+---+---+----+---+-------+
only showing top 20 rows

## Convert data to "features" and "label" format for classifier to accept data
#### Making a pipeline did not work for this. I think because you are only going to fit the model to training data and NOT transform test data

In [4]:
from pyspark.ml.feature import VectorAssembler

In [5]:
assembler = VectorAssembler(inputCols = ["A",
                                        "B",
                                        "C",
                                        "D"],
                           outputCol = "features")

In [6]:
ml_data = assembler.transform(data_df)

In [7]:
# build classifier

random_forest_classifier = RandomForestClassifier(labelCol = "Spoiled", 
                                       featuresCol = "features",
                                      numTrees=100)

In [8]:
# train the classifier with the data. However, we are not tranforming a test set
# because we are not making any predictions. We are just looking at the features
# and figuring out which is most important for the problem.

trained_model = random_forest_classifier.fit(ml_data)

In [9]:
# Show feature and how important it was. Example, feature 215 was not that imporant. feature 462 had .1128 
# which is much higher than all the others so it's important.
# Not a very good example to show this!

trained_model.featureImportances

SparseVector(4, {0: 0.021, 1: 0.0223, 2: 0.9306, 3: 0.0261})

# Preservative C is the most likely culprit according to this model